# Tables creation

### Import libraries

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

### Data path and DB connection

In [ ]:
DATA_PATH = '../data/'

In [ ]:
conn = sqlite3.connect(DATA_PATH + 'technicalDebtDataset.db')

### Fetch main tables

In [ ]:
query = ''' SELECT
                gc.projectID, gc.commitHash, gc.commitMessage,
                sm.complexity, sm.lines, sm.commentLines, sm.duplicatedLines,
                sm.violations, sm.blockerViolations, sm.criticalViolations, sm.majorViolations, sm.minorViolations,
                sm.bugs, sm.codeSmells 
            FROM
                GIT_COMMITS as gc
            INNER JOIN
                SONAR_MEASURES as sm
            WHERE
                gc.commitHash = sm.commitHash
'''

### Show dataframe

In [ ]:
df = pd.read_sql_query(query, conn)
df.head()

### Convert some columns to numeric

In [ ]:
cols = [i for i in df.columns if i not in ['projectID','commitHash','commitMessage']]
for col in cols:
    df[col] = df[col].astype(int)

In [ ]:
df.dtypes

### Columns metrics

In [ ]:
df.describe()

### Preprocessing on "violations" types

In [ ]:
fig , ax = plt.subplots(2,2, figsize=(20,6))
plt.subplots_adjust(wspace=0.1, hspace=0.5)
ax[0,0].hist(df['blockerViolations'], bins=100)
ax[0,1].hist(df['criticalViolations'], bins=100)
ax[1,0].hist(df['majorViolations'], bins=100)
ax[1,1].hist(df['minorViolations'], bins=100)

ax[0,0].title.set_text('blockerViolations')
ax[0,0].set_ylabel('Freq')
ax[0,0].set_xlabel('# violations')
ax[0,1].title.set_text('criticalViolations')
ax[0,1].set_ylabel('Freq')
ax[0,1].set_xlabel('# violations')
ax[1,0].title.set_text('majorViolations')
ax[1,0].set_xlabel('# violations')
ax[1,0].set_ylabel('Freq')
ax[1,1].title.set_text('minorViolations')
ax[1,1].set_xlabel('# violations')
ax[1,1].set_ylabel('Freq')
plt.xticks(rotation='vertical')
plt.show()

Cut violations only where there is data.

In [ ]:
df['blockerViolations'] = df['blockerViolations'].where(df['blockerViolations'] <= 100, 100)
df['majorViolations'] = df['majorViolations'].where(df['majorViolations'] <= 10000, 10000)
df['minorViolations'] = df['minorViolations'].where(df['minorViolations'] <= 10000, 10000)

In [ ]:
fig , ax = plt.subplots(2,2, figsize=(20,6))
plt.subplots_adjust(wspace=0.1, hspace=0.5)
ax[0,0].hist(df['blockerViolations'], bins=100)
ax[0,1].hist(df['criticalViolations'], bins=100)
ax[1,0].hist(df['majorViolations'], bins=100)
ax[1,1].hist(df['minorViolations'], bins=100)

ax[0,0].title.set_text('blockerViolations')
ax[0,0].set_ylabel('Freq')
ax[0,0].set_xlabel('# violations')
ax[0,1].title.set_text('criticalViolations')
ax[0,1].set_ylabel('Freq')
ax[0,1].set_xlabel('# violations')
ax[1,0].title.set_text('majorViolations')
ax[1,0].set_xlabel('# violations')
ax[1,0].set_ylabel('Freq')
ax[1,1].title.set_text('minorViolations')
ax[1,1].set_xlabel('# violations')
ax[1,1].set_ylabel('Freq')
plt.xticks(rotation='vertical')
plt.show()

We merge minor and major violations into one, by adding them up.

In [ ]:
df = df.assign(mimaViolations = df['minorViolations'] + df['majorViolations'])

In [ ]:
fig , ax = plt.subplots(2,2, figsize=(20,6))
plt.subplots_adjust(wspace=0.1, hspace=0.5)

ax[0,0].hist(df['blockerViolations'], bins=100)
ax[0,1].hist(df['criticalViolations'], bins=100)
ax[1,0].hist(df['mimaViolations'], bins=100)

ax[0,0].title.set_text('blockerViolations')
ax[0,0].set_ylabel('Freq')
ax[0,0].set_xlabel('# violations')
ax[0,1].title.set_text('criticalViolations')
ax[0,1].set_ylabel('Freq')
ax[0,1].set_xlabel('# violations')
ax[1,0].title.set_text('mimaViolations')
ax[1,0].set_xlabel('# violations')
ax[1,0].set_ylabel('Freq')
plt.xticks(rotation='vertical')
plt.show()

Create bins for different groups.

In [ ]:
# Partition by steps, intervals of the same size
def categorize_1(N_BINS, v):
    bins = np.arange(min(v), max(v)+1, step = max(v)//N_BINS)
    bins[-1] += 1
    bins[0] = -1
    return bins

# Partition according to the distribution of the data, 
# same number of items in each interval
def categorize_2(N_BINS, v):
    v = v.sort_values()
    n = len(v)
    id_seps = [int(i*n/N_BINS) for i in range(0,N_BINS+1)]
    seps = [v.iloc[el-1] for el in id_seps]
    seps[0] = v.iloc[0]-1
    return seps

In [ ]:
v = df.blockerViolations
n_groups = 3

categorize_2(n_groups, v)

In [ ]:
v = df.blockerViolations
n_groups = 3

categorize_1(n_groups, v)

Create interval variables for each violation.

In [ ]:
n_groups = 2

In [ ]:
df['bin_blockerViolations'] = pd.cut(df['blockerViolations'], categorize_2(n_groups, df['blockerViolations']))
df['bin_criticalViolations'] = pd.cut(df['criticalViolations'], categorize_2(n_groups, df['criticalViolations']))
df['bin_mimaViolations'] = pd.cut(df['mimaViolations'], categorize_2(n_groups, df['mimaViolations']))

In [ ]:
df['class_blockerViolations'] = df['bin_blockerViolations'].cat.codes.astype(str)
df['class_criticalViolations'] = df['bin_criticalViolations'].cat.codes.astype(str)
df['class_mimaViolations'] = df['bin_mimaViolations'].cat.codes.astype(str)

Merge them all and create a colum paste of all combination of classes.

In [ ]:
df["class"] = df["class_blockerViolations"] + df["class_criticalViolations"] + df["class_mimaViolations"]
df['class']

Assign a category to each combination of class.

In [ ]:
df['class'] = df['class'].astype('category')
df['category'] = df['class'].cat.codes
df[['class','category']]

We get the combination categories below.

In [ ]:
print(sorted(df['class'].unique()))

Category codification into integers

In [ ]:
print(sorted(df['category'].unique()))

Check if there are NA categories.

In [ ]:
df[df.category.isna()]

We create a lookup table of the data created (in order to check which intervals where chosen, if needed)

In [ ]:
viols = ['bin_blockerViolations','bin_criticalViolations','bin_mimaViolations']
categories = pd.DataFrame()
for v in viols:
    t = pd.DataFrame({'interval': df[v].unique()})
    t['type'] = v
    t['class'] = np.arange(0,len(df[v].unique()))
    categories = pd.concat([categories,t])
categories.reset_index(drop=True)

### Resulting table

In [ ]:
df

Number of instances for each category

In [ ]:
df['category'].value_counts(), df['category'].count(), 

Category distribution plot

In [ ]:
import seaborn as sns
fig , ax = plt.subplots(1,1, figsize=(15,5))
sns.histplot(df['category'], bins=100, ax =ax)
ax.set_xticks(df.category.unique())
ax.set_xlabel("Category")
ax.set_ylabel("Count [log scale]")
ax.set_title('Histogram of values per category')
#ax.set_yscale('log')
plt.show()

### Save table data

Save it as pickle.

In [ ]:
SAVE_PATH = '../data/our_data/'

In [ ]:
filename = 'commits_violations_8.pkl'

df.to_pickle(SAVE_PATH + filename)

We can read it by using:

In [ ]:
df = pd.read_pickle(SAVE_PATH + filename)
df.head()

### Pearson Correlation of Features

In [ ]:
dff = df[['complexity', 'lines', 'commentLines', 'duplicatedLines', 'violations','category']]

In [ ]:
colormap = plt.cm.viridis
plt.figure(figsize=(10,10))
plt.title('Pearson Correlation of Features', size=15)

sns.heatmap(dff.astype(float).corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)